In [33]:
from scipy.fftpack import fft, fftfreq
import os
import matplotlib.pyplot as plt
import numpy as np
import librosa.display
import librosa
import soundfile as sf                                                      
import sklearn
import pickle
from sklearn.svm import LinearSVC
import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import pickle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import utils
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics


In [34]:
bpm = []
# grabs bpm and S data files given the directory
# containing the pkl files respectively
dir = "pkl_data_files/"

#This one grabs bpm
for x in os.listdir(dir):
    if x.endswith("_bpm.pkl"):
        with open(str(dir + x) , 'rb') as fp:
            file_bpm = pickle.load(fp)
            file_bpm = np.round(file_bpm)
            # print(len(bpm),bpm)
            bpm.extend(file_bpm)
bpm = np.array(bpm)
bpm.shape, bpm

((75,),
 array([ 88.,  88.,  88.,  88.,  88., 102., 102., 102., 102., 102., 117.,
        102., 102., 102., 102., 102., 102., 102.,  73.,  88., 117., 102.,
        102., 102., 102., 102., 102., 102.,  88.,  59.,  88., 117.,  88.,
         73.,  73., 102., 102.,  73.,  73.,  59.,  73., 102., 117., 102.,
         88.,  59.,  73.,  73.,  88.,  88.,  59.,  73.,  73.,  59.,  73.,
         73.,  73.,  88.,  88., 102., 117., 117., 117., 117., 117., 117.,
        117., 117., 117., 117., 117., 117., 132., 132., 117.]))

In [35]:
# bpm_rand = np.random.randint(-10, high=10, size=(len(bpm)))
# bpm += bpm_rand
# bpm

In [36]:
S = []
S_60 = []

# This one grabs S
for x in os.listdir(dir):
    if x.endswith("S_average_values.pkl"):
        with open(str(dir + x) , 'rb') as fp:
            file_S = pickle.load(fp)
            print(len(file_S),file_S) 

            arr = []
            for count, s in enumerate(file_S):
                if count%120 == 0 and count!=0:
                    S_60.append(arr)
                    arr = []
                arr.append(s)
len(S_60)

1915 [ 4.38995305e-06  4.24483610e-06  4.70390250e-06 ... -2.18372966e-06
 -2.83716907e-06 -2.37856497e-06]
1547 [-2.02479840e-05 -1.95322119e-05 -1.67501185e-05 ... -3.04877601e-05
 -2.85369876e-05 -2.77349427e-05]
1989 [-1.56515338e-06 -2.03707953e-06 -1.38745120e-06 ... -1.63695606e-06
 -1.68137953e-06 -1.96251255e-06]
2024 [3.10732474e-06 3.77809789e-07 3.13322686e-06 ... 2.39981424e-06
 2.29844174e-06 3.34401887e-06]
1930 [5.66288936e-05 5.22966390e-05 5.71886879e-05 ... 2.99108768e-05
 2.71840011e-05 2.70056522e-05]


75

In [37]:
S_60 = np.array(S_60)
bpm = np.array(bpm)
S_60.shape, bpm.shape

((75, 120), (75,))

In [38]:
X = S_60
Y = bpm
X.shape, Y.shape

((75, 120), (75,))

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [40]:
clf = LinearSVC()
clf.fit(X_train,y_train)
y_hat = clf.predict(X_test)

In [41]:
# Will comment out / delete later, but it's a good sanity check
for i in range(len(y_test)):
    print(y_test[i], y_hat[i])

88.0 102.0
117.0 102.0
117.0 102.0
88.0 102.0
102.0 102.0
117.0 102.0
88.0 102.0
102.0 102.0
117.0 102.0
117.0 102.0
73.0 102.0
117.0 102.0
88.0 102.0
73.0 102.0
102.0 102.0
102.0 102.0
102.0 102.0
73.0 102.0
73.0 102.0
117.0 102.0
59.0 102.0
73.0 102.0
102.0 102.0


In [42]:
# Will create a function that better gets accuracy
# def get_acc(y_hat, y_test):
    

In [43]:
print('Testing Accuracy: ', np.sum((y_hat == y_test))/len(y_test))
# print('Testing Accuracy: ', get_acc(y_hat, y_test))

Testing Accuracy:  0.2608695652173913


In [44]:
K = [1,3,5,7]

# loop through k's as well
mean_acc = np.zeros((len(K)-1))
for i, k in enumerate(K):
    # default euclidean distance
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    yhat = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, yhat)    
    print("K = "+str(k)+"; Accuracy: "+str(acc))
    mean_acc[i-1] = acc

print( "The best accuracy was:", np.round(mean_acc.max()*100,2),'%') 

K = 1; Accuracy: 0.43478260869565216
K = 3; Accuracy: 0.2608695652173913
K = 5; Accuracy: 0.2608695652173913
K = 7; Accuracy: 0.43478260869565216
The best accuracy was: 43.48 %
